In [ ]:
!pip install langchain
!pip install sentence-transformers
!pip install chromadb
!pip install bitsandbytes-cuda110 bitsandbytes
!pip install accelerate
!pip install langchain_community

Restart Kernel

In [ ]:
!git clone https://github.com/faridhn2/KFCRAGLLM.git

fatal: destination path 'KFCRAGLLM' already exists and is not an empty directory.


In [ ]:
%cd KFCRAGLLM

/content/KFCRAGLLM


In [ ]:
from huggingface_hub import notebook_login


notebook_login()

In [ ]:
# all in one
import torch
from IPython.display import Markdown, display
from langchain import PromptTemplate
from langchain import HuggingFacePipeline

from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import json

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:

class ChromaShip:
    """
    Class representing a document in the Chroma vector store.
    """
    def __init__(self, content, metadata=None):
        """
        Constructor for ChromaShip class.

        Parameters:
        - content: str, the content of the document.
        - metadata: any, additional metadata associated with the document.
        """
        self.page_content: str = content
        self.metadata = metadata


In [ ]:
with open('menu.json') as f:
        data = json.load(f)

        documents = []
        for key, value in data.items():
            if key == 'Location':
                docs = [f'Location: {value["name"]}']
                documents.extend([ChromaShip(content=doc, metadata=None) for doc in docs])
            elif key == 'Menus':
                for menu_code, detail_menu in value.items():
                    text = f'Menu code : {menu_code} , '
                    text+=f'Menu Name : {detail_menu["name"]} , '
                    text+=f'Menu Price : {detail_menu["price"]} , '
                    text+=f'Menu contents : {detail_menu["contents"]} , '
                    docs= [text]
                    documents.extend([ChromaShip(content=doc, metadata=None) for doc in docs])
            else:

                for item, item_value in value.items():


                    text = f'Item code : {item} , Item Name : {item_value[0]} , Item Price : {item_value[1]} , '
                    detail_data = item_value[2]

                    if 'nutritionalInfo' in detail_data:
                        for detail_key, detail_value in detail_data['nutritionalInfo'].items():
                            text+= 'nutritional info : '
                            if detail_key == 'kcal':
                                text+= f'calories : {detail_value} , '

                            else:
                                text+= f'{detail_key} : {detail_value} , '

                    if 'available' in detail_data:
                        text+= f'available : {["No", "Yes"][int(detail_data["available"])]} , '

                    else:
                        text+= f'available : Yes , '

                    docs = [text]
                    documents.extend([ChromaShip(content=doc, metadata=None) for doc in docs])


In [ ]:
documents[9].page_content

"Item code : D2 , Item Name : Pepsi , Item Price : 2.8 , nutritional info : calories : 170 , nutritional info : fat : 10 , nutritional info : protein : 2 , nutritional info : itemId : 9 , nutritional info : allergens : ['dairy'] , available : No , "

In [ ]:
len(documents)

91

In [ ]:
embeddings = HuggingFaceEmbeddings(
            model_name="thenlper/gte-large",
            model_kwargs={"device": "cuda"},
            encode_kwargs={"normalize_embeddings": True})

db = Chroma.from_documents(documents, embeddings, persist_directory="db_njscuba")


In [ ]:
db.search('Location',search_type='similarity')

[Document(metadata={}, page_content='Item code : D12 , Item Name : Sisi , Item Price : 15 , available : No , '),
 Document(metadata={}, page_content='Item code : D12 , Item Name : Sisi , Item Price : 15 , available : No , '),
 Document(metadata={}, page_content='Item code : D12 , Item Name : Sisi , Item Price : 15 , available : No , '),
 Document(metadata={}, page_content='Item code : D12 , Item Name : Sisi , Item Price : 15 , available : No , ')]

In [ ]:
# q_em = embeddings.embed_query('where is this?')
db.similarity_search_with_score(' 150 calories ',k=3)

[(Document(metadata={}, page_content="Item code : C7 , Item Name : Original Piece , Item Price : 15 , nutritional info : calories : 150 , nutritional info : fat : 100 , nutritional info : protein : 10 , nutritional info : itemId : 76 , nutritional info : allergens : [''] , available : No , "),
  0.31124207377433777),
 (Document(metadata={}, page_content="Item code : C7 , Item Name : Original Piece , Item Price : 15 , nutritional info : calories : 150 , nutritional info : fat : 100 , nutritional info : protein : 10 , nutritional info : itemId : 76 , nutritional info : allergens : [''] , available : No , "),
  0.31124207377433777),
 (Document(metadata={}, page_content="Item code : C7 , Item Name : Original Piece , Item Price : 15 , nutritional info : calories : 150 , nutritional info : fat : 100 , nutritional info : protein : 10 , nutritional info : itemId : 76 , nutritional info : allergens : [''] , available : No , "),
  0.31124207377433777)]

In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"


quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True
        )

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
1000.000000000000000000001

1000.0

In [ ]:
text_generation_pipeline = pipeline(
            model=model,
            tokenizer=tokenizer,
            task="text-generation",
            temperature=0.2,
            do_sample=True,
            repetition_penalty=1.1,
            return_full_text=False,
            max_new_tokens=400,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,)

In [ ]:
text_generation_pipeline('Who is Elon Musk')

[{'generated_text': 'ان\n\n# Ilan Maskiyan\n\nIlan Maskiyan is a painter, sculptor and installation artist. He was born in Tehran, Iran, in 1978. He studied painting at the Faculty of Fine Arts, University of Tehran, from 1996 to 2003. In 2004 he moved to Paris, where he continued his artistic studies at the École des Beaux-Arts de Paris, under the supervision of professors such as Alain Le Quernec, Jacques Rancière, and Christian Boltanski.\n\nMaskiyan\'s work is characterized by its exploration of the relationship between the individual and society, and the role of art in shaping this relationship. His paintings often depict scenes of everyday life, but with a surreal or dreamlike quality that invites the viewer to question their assumptions about reality. His sculptures and installations are similarly thought-provoking, using unconventional materials and unexpected juxtapositions to challenge our perceptions of the world around us.\n\nOne of Maskiyan\'s most notable works is "The Wa

In [ ]:
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
            llm=llm, chain_type="stuff", retriever=db.as_retriever())

In [ ]:
q = 'Please recommend an item for a meat lover '
qa_chain.run(q)

" Based on the given information, I cannot recommend the Veggie Burger as it is not available and it is a vegetarian item. For a meat lover, I would suggest looking into other items with meat in them. Without more specific information about the meat lover's preferences, I cannot provide a specific recommendation. However, some popular options might include a beef burger, chicken sandwich, or pork ribs."

In [ ]:


class KFCRAG():
    """
    Class for KFCRAG functionality.
    """
    def __init__(self, json_file='menu.json'):
        """
        Constructor for KFCRAG class.

        Parameters:
        - json_file: str, the path to the JSON file containing data.
        """
        self.json_file = json_file
        self.load_json()
        self.embed()
        self.create_rag()
        self.create_pipeline()
        self.create_llm()
        self.create_qa_chain()

    def load_json(self):
        """
        Load data from a JSON file and prepare documents for Chroma vector store.
        """
        with open(self.json_file) as f:
            data = json.load(f)

        self.documents = []
        for key, value in data.items():
            if key == 'Location':
                docs = [f'Location: {value["name"]}']
                self.documents.extend([ChromaShip(content=doc, metadata=None) for doc in docs])
            elif key == 'Menus':
                for menu_code, detail_menu in value.items():
                    text = f'Menu code : {menu_code} , '
                    text+=f'Menu Name : {detail_menu["name"]} , '
                    text+=f'Menu Price : {detail_menu["price"]} , '
                    text+=f'Menu contents : {detail_menu["contents"]} , '
                    docs= [text]
                    self.documents.extend([ChromaShip(content=doc, metadata=None) for doc in docs])
            else:

                for item, item_value in value.items():


                    text = f'Item code : {item} , Item Name : {item_value[0]} , Item Price : {item_value[1]} , '
                    detail_data = item_value[2]

                    if 'nutritionalInfo' in detail_data:
                        for detail_key, detail_value in detail_data['nutritionalInfo'].items():
                            text+= 'nutritional info : '
                            if detail_key == 'kcal':
                                text+= f'calories : {detail_value} , '

                            else:
                                text+= f'{detail_key} : {detail_value} , '

                    if 'available' in detail_data:
                        text+= f'available : {["No", "Yes"][int(detail_data["available"])]} , '

                    else:
                        text+= f'available : Yes , '

                    docs = [text]
                    self.documents.extend([ChromaShip(content=doc, metadata=None) for doc in docs])


    def embed(self):
        """
        Embed documents into the Chroma vector store.
        """
        self.embeddings = HuggingFaceEmbeddings(
            model_name="thenlper/gte-large",
            model_kwargs={"device": "cuda"},
            encode_kwargs={"normalize_embeddings": True})

        self.db = Chroma.from_documents(self.documents, self.embeddings, persist_directory="db_njscuba")

    def create_rag(self):
        """
        Create a RAG (Retrieval-Augmented Generation) model.
        """
        MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True
        )

        self.tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
        self.tokenizer.pad_token = self.tokenizer.eos_token

        self.model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            torch_dtype=torch.float16,
            trust_remote_code=True,
            device_map="auto",
            quantization_config=quantization_config
        )

    def create_pipeline(self):
        """
        Create a text generation pipeline using the RAG model.
        """
        self.text_generation_pipeline = pipeline(
            model=self.model,
            tokenizer=self.tokenizer,
            task="text-generation",
            temperature=0.2,
            do_sample=True,
            repetition_penalty=1.1,
            return_full_text=True,
            max_new_tokens=400,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.eos_token_id,)

    def create_llm(self):
        """
        Create a language model pipeline using the text generation pipeline.
        """
        self.llm = HuggingFacePipeline(pipeline=self.text_generation_pipeline)

    def create_qa_chain(self):
        """
        Create a QA (Question-Answering) chain using the language model and Chroma retriever.
        """
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm, chain_type="stuff", retriever=self.db.as_retriever())

    def query(self, q):
        """
        Run a query on the QA chain.

        Parameters:
        - q: str, the query string.

        Returns:
        - result: any, the result of the query.
        """
        return self.qa_chain.run(q)

if __name__ == '__main__':
    kfc_rag = KFCRAG()
    print(kfc_rag.query('Give me an orange chocolate milkshake, medium'))

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Item code : DE1 , Item Name : Chocolate Sundae , Item Price : 2.5 , nutritional info : calories : 220 , nutritional info : fat : 9 , nutritional info : protein : 3 , nutritional info : itemId : 13 , nutritional info : allergens : ['dairy', 'soy'] , available : No , 

Item code : DE1 , Item Name : Chocolate Sundae , Item Price : 2.5 , nutritional info : calories : 220 , nutritional info : fat : 9 , nutritional info : protein : 3 , nutritional info : itemId : 13 , nutritional info : allergens : ['dairy', 'soy'] , available : No , 

Item code : DE1 , Item Name : Chocolate Sundae , Item Price : 2.5 , nutritional info : calories : 220 , nutritional info : fat : 9 , nutritional info : protein : 3 , nutritional info : itemId : 13 , nutritional info : allergens : ['dairy', 'soy'] , available : No , 

Item code : DE1 , Item Name : Ch

In [ ]:
from IPython.display import Markdown, display

In [ ]:
q = 'Give me a Veggie Tender, medium, with salad'
result = kfc_rag.query(q)
display(Markdown(f"<b>{q}</b>"))
display(Markdown(f"<p>{result}</p>"))

<b>Give me a Veggie Tender, medium, with salad</b>

<p>Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Menu code : M20 , Menu Name : 4 Veggie Tender meal , Menu Price : 30 , Menu contents : [['V2', 4], {'from': 'Drinks', 'size': 'L'}, {'from': 'Side dishes', 'choose': 1}] , 

Menu code : M20 , Menu Name : 4 Veggie Tender meal , Menu Price : 30 , Menu contents : [['V2', 4], {'from': 'Drinks', 'size': 'L'}, {'from': 'Side dishes', 'choose': 1}] , 

Menu code : M20 , Menu Name : 4 Veggie Tender meal , Menu Price : 30 , Menu contents : [['V2', 4], {'from': 'Drinks', 'size': 'L'}, {'from': 'Side dishes', 'choose': 1}] , 

Menu code : M20 , Menu Name : 4 Veggie Tender meal , Menu Price : 30 , Menu contents : [['V2', 4], {'from': 'Drinks', 'size': 'L'}, {'from': 'Side dishes', 'choose': 1}] , 

Question: Give me a Veggie Tender, medium, with salad
Helpful Answer: Based on the given menu codes and contents, the "4 Veggie Tender meal" is already specified as having a large drink and one side dish chosen for it. To order a Veggie Tender with a medium size and a salad instead of a side dish or drink, you would need to find the menu code for the Veggie Tender with medium size and the separate price for a salad. Unfortunately, this information was not provided in the context. Therefore, I cannot provide a definitive answer to your request. However, if you are ordering at the counter, you could ask the cashier for a Veggie Tender with medium size and a salad, and they should be able to give you an accurate price based on their current menu.</p>